In [2]:
from datasketch import MinHash

data1 = ['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'datasets']
data2 = ['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
        'estimating', 'the', 'similarity', 'between', 'documents']

m1, m2 = MinHash(), MinHash()
for d in data1:
    m1.update(d.encode('utf8'))
for d in data2:
    m2.update(d.encode('utf8'))
print("Estimated Jaccard for data1 and data2 is", m1.jaccard(m2))

s1 = set(data1)
s2 = set(data2)
actual_jaccard = float(len(s1.intersection(s2)))/float(len(s1.union(s2)))
print("Actual Jaccard for data1 and data2 is", actual_jaccard)


Estimated Jaccard for data1 and data2 is 0.7109375
Actual Jaccard for data1 and data2 is 0.7142857142857143


In [3]:
from datasketch import MinHash, MinHashLSH

set1 = set(['minhash', 'is', 'a', 'probabilistic', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'datasets'])
set2 = set(['minhash', 'is', 'a', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])
set3 = set(['minhash', 'is', 'probability', 'data', 'structure', 'for',
            'estimating', 'the', 'similarity', 'between', 'documents'])

m1 = MinHash(num_perm=128)
m2 = MinHash(num_perm=128)
m3 = MinHash(num_perm=128)
for d in set1:
    m1.update(d.encode('utf8'))
for d in set2:
    m2.update(d.encode('utf8'))
for d in set3:
    m3.update(d.encode('utf8'))

# Create LSH index
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)
print("Approximate neighbours with Jaccard similarity > 0.5", result)


Approximate neighbours with Jaccard similarity > 0.5 ['m3', 'm2']


In [6]:
import copy
import itertools
import pandas as pd
import random
import time
from datasketch import MinHash, MinHashLSH
from random import randint, seed, choice

In [7]:
def generate_data(output_path):
    """
    :param output_path: 数据保存路径
    :return: 格式为 id \t attr1,attr2,attrn
    """
    # n个属性
    n_attr = 100
    # 属性维度 2-1000个
    n_attr_list = random.sample(range(2, 1000), n_attr)
    # 样本个数
    n_sample = 1000
    ids = []
    attrs = []

    for i in range(n_sample):
        ids.append('id_' + str(i))
        attr = ['attr%d_%d' % (j, random.randint(1, n_attr_list[j])) for j in range(n_attr)]
        attrs.append(attr)
    # 簇个数
    n_cluster = 4
    # 簇最大的大小
    max_cluster_size = 12
    # 随机生成簇大小
    cluster_size = random.sample(range(2, max_cluster_size), n_cluster)
    for i in range(n_cluster):
        for j in range(cluster_size[i]):
            if j == 0:
                attr = ['attr%d_%d' % (j, random.randint(1, n_attr_list[j])) for j in range(n_attr)]
                attrs.append(attr)
            else:
                sim_attr = copy.deepcopy(attr)
                # 随机篡改3-10个元素
                for k in random.sample(range(n_attr), random.randint(3, 10)):
                    sim_attr[k] = sim_attr[k] + str(random.randint(0, 1000))
                attrs.append(sim_attr)
            ids.append('c%d_%d_%d' % (i, cluster_size[i], j))
    df = pd.DataFrame({'id': ids, 'attributes': [','.join(l) for l in attrs]})
    df.to_csv(output_path, index=False, encoding="utf-8", sep='\t')

In [22]:
def merge_set(input):
    input = [list(l) for l in input]

    lenth = len(input)
    position = [-1 for _ in range(lenth)]
    dict1 = {}
    hash = {}
    res = []
    for i in range(lenth):
        for j in input[i]:
            if j not in dict1:  # 如果元素没有出现过，建立元素与所属集合的键值对
                if i not in hash:  # 如果所属集合没有出现过。认为所属集合是父集合，建立连接关系
                    hash[i] = i
                    dict1[j] = i
                else:
                    dict1[j] = i
            else:
                temp = dict1[j]  # 如果元素已经出现过，建立父子集合之间的连接关系
                hash[i] = temp  # 确保一个集合在同时有未出现过元素和已出现过元素的情况下，能够正确与之前集合建立联系
    temphash = {}
    for i in range(lenth):
        j = i
        while hash[j] != j:
            j = hash[j]
        position[i] = j  # 将子集合指向根集合
        if j not in temphash:
            temphash[j] = 1
    for i in range(lenth):
        if position[i] == i:
            continue
        else:
            input[position[i]].extend(input[i])  # 合并集合
            input[position[i]] = list(set(input[position[i]]))  # 合并后去重
    for key in temphash:
        res.append(input[key])  # temphash中存储的都是并查集的父集合（没有上一级的集合），根据temphash生成结果

    return res

In [9]:
hash_num = 100  # 哈希函数个数 约高约精准
band_size = 10  # LSH 哈希桶大小 r
band_num = int(hash_num / band_size)  # LSH 哈希桶的数量

In [10]:
lsh = MinHashLSH(threshold=0.8, num_perm=hash_num, params=(band_num, band_size))

In [11]:
data_path = 'cluster_dataset.txt'

In [12]:
read_chunks = pd.read_csv(data_path, encoding='utf-8', iterator=True, chunksize=10000, delimiter='\t',
                              dtype={'id': str, 'attributes': str})

In [16]:
for chunk in read_chunks:
    chunk.fillna('')
    for id, attribute in zip(chunk.id, chunk.attributes):
        if isinstance(attribute, str):
            attribute_list = attribute.split(',')
            m = MinHash(num_perm=hash_num)
            m.update_batch([s.encode('utf-8') for s in attribute_list])
            lsh.insert(id, m)
        else:
            print('id=%s attr=%s' % (id, attribute))


In [18]:
clusters_list = []
for i, band_i in enumerate(lsh.hashtables):
    for band_hash in band_i:
        if len(band_i[band_hash]) > 1:
            clusters_list.append(band_i[band_hash])
    # 合并集合
clusters_combination = merge_set(clusters_list)
print(clusters_combination)

[['c0_5_0', 'c0_5_3', 'c0_5_4', 'c0_5_1', 'c0_5_2'], ['c1_10_0', 'c1_10_6', 'c1_10_7', 'c1_10_8', 'c1_10_4'], ['c1_10_0', 'c1_10_1', 'c1_10_6', 'c1_10_3', 'c1_10_7', 'c1_10_8', 'c1_10_4', 'c1_10_9', 'c1_10_2'], ['c2_6_1', 'c2_6_5', 'c2_6_4', 'c2_6_3', 'c2_6_0', 'c2_6_2'], ['c3_2_0', 'c3_2_1']]
